# Environment set up
This section ensures all libraries and data are correctly imported.

### Optional GPU check

In [0]:
# Taken from
# https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# Colab only provides one GPU and it is not always guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [0]:
printm()

### Cloning the repository and downloading the data

In [3]:
# Clone repo
!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor

Cloning into 'keras_triplet_descriptor'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 181 (delta 0), reused 1 (delta 0), pack-reused 178
Receiving objects: 100% (181/181), 149.87 MiB | 8.16 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (69/69), done.


In [4]:
# Change directory
%cd /content/keras_triplet_descriptor    


/content/keras_triplet_descriptor


In [0]:
# Download data
!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip


--2019-03-20 23:11:07--  https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 107.152.24.197, 107.152.25.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-03-20 23:11:08--  https://imperialcollegelondon.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Reusing existing connection to imperialcollegelondon.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://imperialcollegelondon.app.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
--2019-03-20 23:11:08--  https://imperialcollegelondon.app.box.com/public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.app.box.com (imperialcolleg

In [0]:
# Extract data
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip

### Importing Necessary Modules

In [0]:
import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate  

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

# Plotting tools
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams["patch.force_edgecolor"] = True

# To download files
from google.colab import files

# Train baseline denoiser

# Train baseline Descriptor on clean patches
To identify the upper bound performance of the baseline descriptor model as well as to gather qualitative insights on the network's output.

## Instantiate baseline descriptor model

### Load clean hpatches data

In [0]:
# Initialise random seeds for reproducibility
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

# Import and read hpatches dataset
hpatches_dir = './hpatches'
splits_path = './splits.json'
splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']
train_fnames = split['train']
test_fnames = split['test']
seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs))


In [0]:
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames, use_clean=True)

### Define model and loss

In [0]:
# Original L2-Net Baseline
def get_baseline_descriptor(shape):
  
  '''Architecture copies HardNet architecture'''
  
  init_weights = keras.initializers.he_normal()
  
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  
  # Final descriptor reshape
  descriptor_model.add(Reshape((128,)))
  
  return descriptor_model

In [0]:
# Original triplet loss
def baseline_triplet_loss(x):
  
  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

### Create online generators

In [0]:
# Initialise random seeds
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

# Define baseline batch size and number of triplets
gen_batch_size = 50
trn_triplets = 100000
val_triplets =  10000

# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=trn_triplets)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=val_triplets)


Using clean patches
100%|██████████| 116/116 [00:33<00:00,  2.82it/s]


100%|██████████| 100000/100000 [00:01<00:00, 69448.02it/s]


Using clean patches
100%|██████████| 116/116 [00:20<00:00,  5.74it/s]


100%|██████████| 10000/10000 [00:00<00:00, 59463.98it/s]


In [0]:
# Plot an example of triplet
plot_triplet(training_generator)

### Compile model

In [0]:
# Initialise random seeds
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

# Define the three input layers
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')

# Create the siamese models
baseline_descriptor_model = get_baseline_descriptor(shape)
ea = baseline_descriptor_model(xa)
ep = baseline_descriptor_model(xp)
en = baseline_descriptor_model(xn)

# Generate the triplet model
loss = Lambda(baseline_triplet_loss)([ea, ep, en])
baseline_descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)

# Compile model: SGD is rejected in favour of Adam even in the baseline model
adm = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00, amsgrad=True)
baseline_descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=adm)

# Initialise lists to record training history
baseline_loss = []
baseline_val = []
baseline_epo = []
bs_epoch_count = 1

In [0]:
# Print a summary of the triplet model
baseline_descriptor_model_trip.summary()

## Train model

### Train and save model - Epoch 1
We obvserve running time and initial loss after the first epoch.

In [0]:
# Train first epoch
print('Epoch number',bs_epoch_count)
baseline_descriptor_hist = baseline_descriptor_model_trip.fit_generator(
                                                    generator=training_generator,
                                                    epochs=1, verbose=1,
                                                    validation_data=val_generator)
baseline_epo.append(bs_epoch_count)
baseline_loss.append(baseline_descriptor_hist.history['loss'])
baseline_val.append(baseline_descriptor_hist.history['val_loss'])
bs_epoch_count += 1

# Save models
baseline_descriptor_model_trip.save('bs_trip_descriptor_e1.h5')
baseline_descriptor_model.save('bs_descriptor_e1.h5')
!curl -F "file=@bs_trip_descriptor_e1.h5" https://file.io
!curl -F "file=@bs_descriptor_e1.h5" https://file.io

('Epoch number', 1)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.2038

100%|██████████| 100000/100000 [00:03<00:00, 27345.47it/s]


2000/2000 [==============================] - 187s 94ms/step - loss: 0.2038 - val_loss: 0.1890
{"success":true,"key":"3TUCLq","link":"https://file.io/3TUCLq","expiry":"14 days"}{"success":true,"key":"ZhdBOs","link":"https://file.io/ZhdBOs","expiry":"14 days"}

### Train and save model - Epoch 5

In [0]:
for e in range(4):
  print('Epoch number',bs_epoch_count)
  
  baseline_descriptor_hist = baseline_descriptor_model_trip.fit_generator(
      generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  
  baseline_epo.append(bs_epoch_count)
  baseline_loss.append(baseline_descriptor_hist.history['loss'])
  baseline_val.append(baseline_descriptor_hist.history['val_loss'])
  bs_epoch_count += 1

# Save models
baseline_descriptor_model_trip.save('bs_trip_descriptor_e5.h5')
baseline_descriptor_model.save('bs_descriptor_e5.h5')
!curl -F "file=@bs_trip_descriptor_e5.h5" https://file.io
!curl -F "file=@bs_descriptor_e5.h5" https://file.io
  
# Print loss curves
print('Epochs',baseline_epo)
print('Loss', baseline_loss)
print('Val',baseline_val)

('Epoch number', 2)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1263

100%|██████████| 100000/100000 [00:03<00:00, 27412.30it/s]


2000/2000 [==============================] - 177s 88ms/step - loss: 0.1262 - val_loss: 0.1562
('Epoch number', 3)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1139

100%|██████████| 100000/100000 [00:03<00:00, 29031.63it/s]


2000/2000 [==============================] - 177s 88ms/step - loss: 0.1139 - val_loss: 0.1298
('Epoch number', 4)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1000

100%|██████████| 100000/100000 [00:04<00:00, 21855.52it/s]


2000/2000 [==============================] - 178s 89ms/step - loss: 0.1000 - val_loss: 0.1145
('Epoch number', 5)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.0897

100%|██████████| 100000/100000 [00:03<00:00, 27346.04it/s]


2000/2000 [==============================] - 178s 89ms/step - loss: 0.0896 - val_loss: 0.1101
{"success":true,"key":"RvLkSP","link":"https://file.io/RvLkSP","expiry":"14 days"}{"success":true,"key":"PqhInI","link":"https://file.io/PqhInI","expiry":"14 days"}('Epochs', [1, 2, 3, 4, 5])
('Loss', [[0.20375348175376712], [0.12624182823907176], [0.11385689811104152], [0.09996190780464895], [0.08964496588223847]])
('Val', [[0.1890304999984801], [0.15621989610459422], [0.12979786690673792], [0.11447668469743803], [0.11011563445208594]])


### Train and save model - Epoch 20

In [0]:
for e in range(15):
  print('Epoch number',bs_epoch_count)
  
  baseline_descriptor_hist = baseline_descriptor_model_trip.fit_generator(
      generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  
  baseline_epo.append(bs_epoch_count)
  baseline_loss.append(baseline_descriptor_hist.history['loss'])
  baseline_val.append(baseline_descriptor_hist.history['val_loss'])
  bs_epoch_count += 1

# Save models
baseline_descriptor_model_trip.save('bs_trip_descriptor_e20.h5')
baseline_descriptor_model.save('bs_descriptor_e20.h5')
!curl -F "file=@bs_trip_descriptor_e20.h5" https://file.io
!curl -F "file=@bs_descriptor_e20.h5" https://file.io
  
# Print loss curves
print('Epochs',baseline_epo)
print('Loss', baseline_loss)
print('Val',baseline_val)

('Epoch number', 6)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.0774

100%|██████████| 100000/100000 [00:03<00:00, 27373.61it/s]


2000/2000 [==============================] - 177s 89ms/step - loss: 0.0774 - val_loss: 0.1096
('Epoch number', 7)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.0748

100%|██████████| 100000/100000 [00:03<00:00, 28913.38it/s]


2000/2000 [==============================] - 176s 88ms/step - loss: 0.0748 - val_loss: 0.1169
('Epoch number', 8)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.0670

...

2000/2000 [==============================] - 181s 91ms/step - loss: 0.0494 - val_loss: 0.0798


100%|██████████| 10000/10000 [00:00<00:00, 61407.77it/s]


('Epoch number', 20)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.0484

100%|██████████| 100000/100000 [00:03<00:00, 27770.80it/s]


2000/2000 [==============================] - 179s 89ms/step - loss: 0.0484 - val_loss: 0.0831
{"success":true,"key":"6otRfC","link":"https://file.io/6otRfC","expiry":"14 days"}{"success":true,"key":"uUJYoA","link":"https://file.io/uUJYoA","expiry":"14 days"}('Epochs', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
('Loss', [[0.20375348175376712], [0.12624182823907176], [0.11385689811104152], [0.09996190780464895], [0.08964496588223847], [0.07738297405181584], [0.07480863291910646], [0.06696327244347049], [0.06532479770145619], [0.06626588492320479], [0.05716229805073635], [0.06180751825054722], [0.05546622675077015], [0.05546771268250086], [0.05212180319649815], [0.05879474075542885], [0.052441509388472696], [0.05406738201301778], [0.0494032579374234], [0.04835177053282405]])
('Val', [[0.1890304999984801], [0.15621989610459422], [0.12979786690673792], [0.11447668469743803], [0.11011563445208594], [0.10961331890663133], [0.11694466379005462], [0.096325404431263

# Train baseline Descriptor on denoised patches
To identify the upper bound performance of the baseline descriptor model as well as to gather qualitative insights on the network's output.

## Instantiate baseline descriptor model

### Load clean hpatches data

In [0]:
# Initialise random seeds for reproducibility
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

# Import and read hpatches dataset
hpatches_dir = './hpatches'
splits_path = './splits.json'
splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']
train_fnames = split['train']
test_fnames = split['test']
seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs))


In [0]:
# Loading images
hPatches = HPatches(train_fnames=train_fnames, denoise_model=trained_denoiser, test_fnames=test_fnames, use_clean=False)

### Define model and loss

In [0]:
# Original L2-Net Baseline
def get_baseline_descriptor(shape):
  
  '''Architecture copies HardNet architecture'''
  
  init_weights = keras.initializers.he_normal()
  
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  
  # Final descriptor reshape
  descriptor_model.add(Reshape((128,)))
  
  return descriptor_model

In [0]:
# Original triplet loss
def baseline_triplet_loss(x):
  
  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

### Create online generators

In [0]:
# Initialise random seeds
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

# Define baseline batch size and number of triplets
gen_batch_size = 50
trn_triplets = 100000
val_triplets =  10000

# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=trn_triplets)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=val_triplets)


In [0]:
# Plot an example of triplet
plot_triplet(training_generator)

### Compile model

In [0]:
# Initialise random seeds
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

# Define the three input layers
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')

# Create the siamese models
baseline_descriptor_model = get_baseline_descriptor(shape)
ea = baseline_descriptor_model(xa)
ep = baseline_descriptor_model(xp)
en = baseline_descriptor_model(xn)

# Generate the triplet model
loss = Lambda(baseline_triplet_loss)([ea, ep, en])
baseline_descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)

# Compile model: SGD is rejected in favour of Adam even in the baseline model
adm = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00, amsgrad=True)
baseline_descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=adm)

# Initialise lists to record training history
baseline_loss = []
baseline_val = []
baseline_epo = []
bs_epoch_count = 1

In [0]:
# Print a summary of the triplet model
baseline_descriptor_model_trip.summary()

## Train model

### Train and save model - Epoch 1
We obvserve running time and initial loss after the first epoch.

In [0]:
# Train first epoch
print('Epoch number',bs_epoch_count)
baseline_descriptor_hist = baseline_descriptor_model_trip.fit_generator(
                                                    generator=training_generator,
                                                    epochs=1, verbose=1,
                                                    validation_data=val_generator)
baseline_epo.append(bs_epoch_count)
baseline_loss.append(baseline_descriptor_hist.history['loss'])
baseline_val.append(baseline_descriptor_hist.history['val_loss'])
bs_epoch_count += 1

# Save models
baseline_descriptor_model_trip.save('dnbs_trip_descriptor_e1.h5')
baseline_descriptor_model.save('dnbs_descriptor_e1.h5')
!curl -F "file=@dnbs_trip_descriptor_e1.h5" https://file.io
!curl -F "file=@dnbs_descriptor_e1.h5" https://file.io

('Epoch number', 1)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.2213

100%|██████████| 100000/100000 [00:04<00:00, 22832.47it/s]


2000/2000 [==============================] - 176s 88ms/step - loss: 0.2212 - val_loss: 0.2010
{"success":true,"key":"LcKSLr","link":"https://file.io/LcKSLr","expiry":"14 days"}{"success":true,"key":"PtiqPP","link":"https://file.io/PtiqPP","expiry":"14 days"}

In [0]:
bs_epoch_count

1

### Train and save model - Epoch 5

In [0]:
for e in range(4):
  print('Epoch number',bs_epoch_count)
  
  baseline_descriptor_hist = baseline_descriptor_model_trip.fit_generator(
      generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  
  baseline_epo.append(bs_epoch_count)
  baseline_loss.append(baseline_descriptor_hist.history['loss'])
  baseline_val.append(baseline_descriptor_hist.history['val_loss'])
  bs_epoch_count += 1

# Save models
baseline_descriptor_model_trip.save('dnbs_trip_descriptor_e5.h5')
baseline_descriptor_model.save('dnbs_descriptor_e5.h5')
!curl -F "file=@dnbs_trip_descriptor_e5.h5" https://file.io
!curl -F "file=@dnbs_descriptor_e5.h5" https://file.io
  
# Print loss curves
print('Epochs',baseline_epo)
print('Loss', baseline_loss)
print('Val',baseline_val)

('Epoch number', 2)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1541

 94%|█████████▍| 94066/100000 [00:03<00:00, 31169.07it/s]

2000/2000 [==============================] - 171s 86ms/step - loss: 0.1541 - val_loss: 0.1838


100%|██████████| 10000/10000 [00:00<00:00, 16706.57it/s]


('Epoch number', 3)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1389

...

('Epoch number', 5)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1173

 93%|█████████▎| 92627/100000 [00:03<00:00, 30022.26it/s]

2000/2000 [==============================] - 171s 86ms/step - loss: 0.1173 - val_loss: 0.1497


100%|██████████| 10000/10000 [00:00<00:00, 60729.72it/s]


{"success":true,"key":"kKE9pH","link":"https://file.io/kKE9pH","expiry":"14 days"}{"success":true,"key":"cpWDMC","link":"https://file.io/cpWDMC","expiry":"14 days"}('Epochs', [1, 2, 3, 4, 5])
('Loss', [[0.22122364778607154], [0.15412222058465705], [0.13891368479607627], [0.12695958790479928], [0.11728867068770342]])
('Val', [[0.20100265827029942], [0.18383058869279922], [0.17476918146945536], [0.15307056670077146], [0.1497141424147412]])


### Train and save model - Epoch 20

In [0]:
for e in range(15):
  print('Epoch number',bs_epoch_count)
  
  baseline_descriptor_hist = baseline_descriptor_model_trip.fit_generator(
      generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  
  baseline_epo.append(bs_epoch_count)
  baseline_loss.append(baseline_descriptor_hist.history['loss'])
  baseline_val.append(baseline_descriptor_hist.history['val_loss'])
  bs_epoch_count += 1

# Save models
baseline_descriptor_model_trip.save('dnbs_trip_descriptor_e20.h5')
baseline_descriptor_model.save('dnbs_descriptor_e20.h5')
!curl -F "file=@dnbs_trip_descriptor_e20.h5" https://file.io
!curl -F "file=@dnbs_descriptor_e20.h5" https://file.io
  
# Print loss curves
print('Epochs',baseline_epo)
print('Loss', baseline_loss)
print('Val',baseline_val)

('Epoch number', 6)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1046

 83%|████████▎ | 83289/100000 [00:03<00:00, 32620.92it/s]

2000/2000 [==============================] - 173s 86ms/step - loss: 0.1046 - val_loss: 0.1521


100%|██████████| 10000/10000 [00:00<00:00, 12004.43it/s]


('Epoch number', 7)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1021

...

('Epoch number', 20)
Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.0669

100%|█████████▉| 99935/100000 [00:03<00:00, 27327.59it/s]

2000/2000 [==============================] - 171s 86ms/step - loss: 0.0670 - val_loss: 0.1201


100%|██████████| 10000/10000 [00:00<00:00, 57083.15it/s]


{"success":true,"key":"AI90R3","link":"https://file.io/AI90R3","expiry":"14 days"}{"success":true,"key":"p66azi","link":"https://file.io/p66azi","expiry":"14 days"}('Epochs', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
('Loss', [[0.22122364778607154], [0.15412222058465705], [0.13891368479607627], [0.12695958790479928], [0.11728867068770342], [0.10458592997015512], [0.10210923682362046], [0.0906655819955522], [0.08820582015627587], [0.08229035005806509], [0.083896127131502], [0.08455030574434204], [0.08364793859165912], [0.07722409284795503], [0.07227165060596781], [0.07292610311981435], [0.07321893805771833], [0.06962838557911163], [0.07236122024823453], [0.06695349532148248]])
('Val', [[0.20100265827029942], [0.18383058869279922], [0.17476918146945536], [0.15307056670077146], [0.1497141424147412], [0.15210309472400696], [0.15220730077475308], [0.13823029227554798], [0.13256005036644639], [0.16778525381349027], [0.1578529431950301], [0.13574334086384623], [